In [2]:
!pip install pandas

Начем с нахождения реплики приветствия и прощания, поскольку принцип их нахождения идентичен.
Тк список реплик с приветствием и прощанием является словарным, просто выпишем все варианты (даже самые странные):

In [1]:
russian_greetings = '''В кои-то веки
Вот так встреча
Всегда рады Вам
Глубокое почтение
Глубокое глубочайшее
Горячий привет
Горячо приветствую
Доброго здоровья
Доброго здоровьица
Доброе утро
Добро пожаловать
Добрый вечер
Добрый день
Дозвольте приветствовать
Душевно рад
Душою рад Вас видеть
Желаю здравствовать
Здравия желаю
Здравствуйте
Здравствуй
Какая встреча
Какие гости
Моё почтение
Нижайшее почтение
Позвольте Вас приветствовать
Почитаю приятным долгом засвидетельствовать Вам моё почтение
Почитаю приятным долгом засвидетельствовать Вам моё уважение
Почтение моё
Привет
Приветствую Вас
Приветствую Вас от имени
приветствует
Приятный вечер
Приятный день
Рад Вам
Рад Вас видеть
Рад Вас видеть в добром здравии
Рад Вас приветствовать
Рад Вас слышать
Рад пожать Вашу руку
Разрешите Вас приветствовать
Разрешите засвидетельствовать Вам моё почтение
Разрешите засвидетельствовать Вам моё уважение
С возвращением
С выздоровлением
С добрым утром
Сердечно приветствую Вас
Сердечно рад Вам
Сердечный поклон Вам
Сердечный привет Вам
Сколько лет, сколько зим
Тысячу лет Вас не видел
Тысячу лет Вас не виделись'''
russian_farewell = '''Счастливо
прощайте
до встречи
до свидания
всего доброго
всего хорошего
пока
до свидания
ещё увидимся
до скорой встречи
всего хорошего
счастливо
до завтра
доброго пути
будьте здоровы
всегда рады вас видеть
приятно было познакомиться
всего доброго
счастливого пути
счастливо оставаться
до вечера
до утра
надеюсь ещё раз увидеться с вами
до новой встречи
всегда рады вас видеть
будьте здоровы
разрешите откланяться
доброй ночи
чао'''
russian_greetings = russian_greetings.lower().split('\n')
russian_farewell = russian_farewell.lower().split('\n')

Теперь выполним поиск по текстам

In [2]:
import pandas as pd

df = pd.read_csv('test_data.csv')


def greeting_detection(text):
    text = text.lower()
    for greeting in russian_greetings:
        if greeting in text:
            return True
    return False


def farewell_detection(text):
    text = text.lower()
    for goodbye in russian_farewell:
        if goodbye in text:
            return True
    return False


df['manager_greeting'] = df[df['role'] == 'manager']['text'].apply(greeting_detection)
df['manager_farewell'] = df[df['role'] == 'manager']['text'].apply(farewell_detection)


Предварительно проверим нашу работу. Реплика с приветствием будет в начале, а c прощанием в конце, проверим это.

In [3]:
df.head()

,dlg_id,line_n,role,text,manager_greeting,manager_farewell
0,0,0,client,Алло,NaN,NaN
1,0,1,manager,Алло здравствуйте,True,False
2,0,2,client,Добрый день,NaN,NaN
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...,False,False
4,0,4,client,Ага,NaN,NaN


In [4]:
df.tail()

,dlg_id,line_n,role,text,manager_greeting,manager_farewell
475,5,138,manager,По поводу виджетов и с ними уже обсудите конкр...,False,False
476,5,139,manager,Все я вам высылаю счет и с вами на связи если ...,False,False
477,5,140,client,Спасибо спасибо,NaN,NaN
478,5,141,client,Да да тогда созвонимся ага спасибо вам давайте,NaN,NaN
479,5,142,manager,Ну до свидания хорошего вечера,False,True


Теперь приступим к более сложной задаче, а находить реплики, где менеджер представил себя. Для этого воспользуемся NER от SpiCy

In [5]:
!pip install -U pip setuptools wheel
!pip install -U spacy
!python -m spacy download ru_core_news_lg

OMP: Error #15: Initializing libomp.dylib, but found libomp.dylib already initialized.
OMP: Hint This means that multiple copies of the OpenMP runtime have been linked into the program. That is dangerous, since it can degrade performance or cause incorrect results. The best thing to do is to ensure that only a single OpenMP runtime is linked into the process, e.g. by avoiding static linking of the OpenMP runtime in any library. As an unsafe, unsupported, undocumented workaround you can set the environment variable KMP_DUPLICATE_LIB_OK=TRUE to allow the program to continue to execute, but that may cause crashes or silently produce incorrect results. For more information, please see http://openmp.llvm.org/


Заметим, что перед представлением используются ключевые слова и выпишем их. Далее будем искать их в тексте и если после них стоит имя, которое мы определяем с помощью NER, то считаем его за имя менеджера.

In [ ]:
import spacy

nlp = spacy.load("ru_core_news_lg")

my_name_is = ['меня зовут', 'зовут меня', 'я', 'хочу представиться', 'мое имя', 'разрешите представиться',
              'давайте знакомиться', 'зовите меня', 'познакомимся', 'меня величают']

def name_detection(text):
    doc = nlp(text)
    text_before = ''
    mni_flag = False
    for token in doc:
        for mni in my_name_is:
            if mni in text_before:
                mni_flag = True
        if token.ent_type_ == 'PER' and mni_flag:
            return token.text
        text_before += token.text
    return ''


df['manager_name'] = df[df['role'] == 'manager']['text'].apply(name_detection)

Проверяем нашу работу
Очевидно, что там где мы определили имя менеджера это и есть предложение, где он представился.
Теперь определим в каждом ли чате менеджер поздоровался и попрощался с клиентом

In [71]:
df[(df['role'] == 'manager') & (df['manager_greeting'] == True) | (df['manager_farewell'] == True)].groupby('dlg_id').agg({'manager_farewell': 'max', 'manager_greeting': 'max'})

,manager_farewell,manager_greeting
dlg_id,,
0,True,True
1,True,True
2,False,True
3,True,True
4,True,False
5,True,False


К сожалению, название компании достать не получилось, тк даже самые точные ner модели очень плохо понимают сущности в нижнем регистре. А искать просто ключевое слово компания как-то глупо, тем более, что неизвестно кол-во слов.